In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from utils import *
import json 

save_path = '/scratch/ds5749/prob-and-stats/dl/'
train_data = read_file('train.txt')
train_data_int = [ [int(i[1:-1]) for i in line]  for line in train_data ]

#get names of the id
products = pd.read_csv('/scratch/ds5749/prob-and-stats/products.csv')
id2name_mapping = dict(zip(products['product_id'], products['product_name']))
name2id_mapping = {value: key for key, value in id2name_mapping.items()}
train_data_names = [ [id2name_mapping[i] for i in line]  for line in train_data_int ]

distinct_count = 500
items, data = filter_file(train_data_names, n_items=distinct_count)
train, val = split_list(data, 0.9)

#save data mapping
flat_list =  [item for sublist in data for item in sublist]
mapping = {item: i for i, item in enumerate(set(flat_list))}
with open(f"{save_path}/item_map{distinct_count}.json", "w") as outfile:
                json.dump(mapping, outfile)
        

In [2]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", 
                                                           num_labels= distinct_count,
                                                           output_attentions = False,
                                                          output_hidden_states = False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [3]:
# max_len= 0 
# for sent in train_data_names:
#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)
#     # Update the maximum sentence length.
#     max_len = max(max_len, len(input_ids))
# print('Max sentence length: ', max_len)

In [4]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import random 
class ItemDataset(Dataset):
    def __init__(self, df, tokenizer, mapping, max_len=512, shuffle=False):
        self.df = df
        self.shuffle = shuffle
        self.max_len= max_len
        self.tokenizer = tokenizer
        self.mapping = mapping

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # row = self.df.iloc[index]
        try:
            label = self.df[index][-1]
        except:
            print(self.df[index])
        text = self.df[index][0:-1]
        if self.shuffle:
            random.shuffle(text)
        text = ','.join(text)
        encoded_input = self.tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        pad_to_max_length = True,
                        max_length = self.max_len,     
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation=True,
                   )
        input_ids, attention_mask = torch.tensor(encoded_input['input_ids']), torch.tensor(encoded_input['attention_mask'])
        one_hot_encoded_label = torch.tensor([0] * len(self.mapping))
        print(len(one_hot_encoded_label))
        one_hot_encoded_label[self.mapping[str(label)]] = 1
        
        # return input_ids, attention_mask, torch.tensor(self.mapping[str(label)])
        return input_ids, attention_mask, one_hot_encoded_label

In [5]:
from torch.utils.data import Dataset, DataLoader
batch_size = 1
dataset = ItemDataset(train, tokenizer, mapping=mapping, max_len=147)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [6]:
from transformers import get_linear_schedule_with_warmup, AdamW

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss()

/ext3/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
model.train()
model.to(device)
for epoch_i in range(0, epochs):
    total_train_loss = 0
    for i, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask ,labels = torch.tensor(input_ids).to(device), torch.tensor(attention_mask).to(device), torch.tensor(labels).to(device)
        # input_ids, attention_mask= torch.tensor(input_ids).to(device), torch.tensor(attention_mask).to(device)
        # labels = labels.unsqueeze(0)
        model.zero_grad()  
        
        outs = model(input_ids, attention_mask=attention_mask)
        logits =  outs.logits
        loss = criterion(logits,labels.float())

        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

/ext3/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/state/partition1/job-32995369/ipykernel_818152/38739347.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids, attention_mask ,labels = torch.tensor(input_ids).to(device), torch.tensor(attention_mask).to(device), torch.tensor(labels).to(device)


500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500


../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], thread: [37,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [22,0,0], t

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
